In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("/usr/local/lib/python3.6/site-packages/libsvm/")
from svmutil import *

In [19]:
X_train = pd.read_csv('X_train.csv',header=None).values
Y_train = pd.read_csv('Y_train.csv',header=None).values.reshape(-1)
X_test = pd.read_csv('X_test.csv',header=None).values
Y_test = pd.read_csv('Y_test.csv',header=None).values.reshape(-1)

In [35]:
kernels = ['linear','polynomial','RBF']
for i in range(3):
    print('Kernel: '+kernels[i])
    params = '-t '+ str(i)
    model = svm_train(Y_train,X_train,params)
    svm_predict(Y_test,X_test, model)

Kernel: linear
Accuracy = 95.08% (2377/2500) (classification)
Kernel: polynomial
Accuracy = 34.68% (867/2500) (classification)
Kernel: RBF
Accuracy = 95.32% (2383/2500) (classification)
